In [ ]:
import gensim
import sklearn
from sklearn.externals import joblib
from sklearn import svm

In [ ]:
model_russian = gensim.models.KeyedVectors.load_word2vec_format('vectors/wiki.ru.vec')

In [ ]:
del model

In [225]:

model = gensim.models.KeyedVectors.load_word2vec_format('vectors/GoogleNews-vectors-negative300.bin', binary=True)

TypeError: load() got an unexpected keyword argument 'binary'

In [204]:
def vectorize_word(word):
	if word in model.wv:
		return pd.Series(model.wv[word])
	else:
		# The pretrained word2vec model has dimensionality 300
		return pd.Series([np.nan] * 300)
def merge_vectors(v1, v2, method):
	print "method", method
	# Concat
	if method == 'concat':
		return pd.concat([v1, v2, y], axis = 1)
	elif method == 'diff':
		# Roller 2014 says to normalize the difference
		diff = v1 - v2
		return pd.concat([diff, y], axis = 1)
	elif method == 'asym':
		print "asym"
		# diff
		a = v1 - v2
		# squared diff - can't tell if they mean the mag^2 or ea element sq?
		b = pd.DataFrame(np.sqrt(np.square(a.values).sum(axis=1)))
		return pd.concat([a, b, y], axis = 1)
	elif method == 'cosine':
		#this fails???
		print "cosine"
		cos = pd.DataFrame(cosine_similarity(vectors_0.values, vectors_1.values).diagonal())
		return pd.concat([v1, v2, cos, y], axis = 1)

In [205]:
folder = 'baroni2012'
method = 'concat'
for subset in ['test', 'train', 'val']:
    path_to_dataset = 'lexical_entailment/' + folder + '/data_lex_' + subset + '.tsv'
    path_to_vectorized = 'lexical_entailment/' + folder + '/data_lex_' + subset + '_vectorized.tsv'
    df = pd.read_csv(path_to_dataset, sep='\t', header=None)
    words_0 = df[0]
    words_1 = df[1]
    y = df[[2]]
    vectors_0 = words_0.apply(vectorize_word)
    vectors_1 = words_1.apply(vectorize_word)
    vectors_x = merge_vectors(vectors_0, vectors_1, method)
    vectors_x.to_csv(path_to_vectorized, sep='\t', header=False, index=False)

method concat
method concat
method concat


In [206]:
from sklearn.externals import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

clf = joblib.load('models/svm.pkl') 

folder = 'baroni2012'
test_vectorized = pd.read_csv('lexical_entailment/'+folder+'/data_lex_test_vectorized.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/'+folder+'/data_lex_val_vectorized.tsv', sep='\t', header=None)

for test_name, test_df in zip(['test', 'val'], [test_vectorized, val_vectorized]):
	orig_rows, orig_cols = test_df.shape

	# Remove rows with NaN
	test_df.dropna(axis=0, inplace=True)

	# Count number of rows removed
	diff = orig_rows - test_df.shape[0]

	X = test_df.iloc[:, :-1]
	y = test_df.iloc[:, -1]

	preds = clf.predict(X)

	true_count = y.sum()
	preds_count = preds.sum()

	print "precision", metrics.precision_score(y, preds)
	print "recall", metrics.recall_score(y, preds)
	print "f1", metrics.f1_score(y, preds)
	print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
	print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])
	print true_count, preds_count, orig_rows

	num_correct = metrics.accuracy_score(y, preds, normalize=False)

	print test_name, ": percentage non-nan correct:", num_correct/float(test_df.shape[0]) 
	print test_name, ": percentage correct overall", num_correct/float(orig_rows)
    

precision 0.71186440678
recall 0.158490566038
f1 0.259259259259
True 0.158490566038
False 0.93536121673
265 59 536
test : percentage non-nan correct: 0.545454545455
test : percentage correct overall 0.537313432836
precision 0.947368421053
recall 0.409090909091
f1 0.571428571429
True 0.409090909091
False 0.976744186047
44 19 87
val : percentage non-nan correct: 0.689655172414
val : percentage correct overall 0.689655172414


In [207]:
train = pd.read_csv('lexical_entailment/bless2011/data_lex_train_vectorized_asym.tsv', sep='\t', header=None)
train.dropna(axis=0, inplace=True)
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

clf = svm.LinearSVC(class_weight='balanced')
clf.fit(X, y)

LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [208]:
test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized_asym.tsv', sep='\t', header=None)
test = pd.read_csv('lexical_entailment/bless2011/data_lex_test.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_val_vectorized_asym.tsv', sep='\t', header=None)

In [216]:
train_df = val_vectorized
test_df = test_vectorized
train_df.dropna(axis=0, inplace=True)
test_df.dropna(axis=0, inplace=True)
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]

In [222]:
val_vectorized

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,300,301
0,-0.189951,0.792969,0.274010,-0.191406,-0.094727,-0.271973,-0.330566,0.040039,0.878906,0.315613,...,0.244141,-0.003906,0.444336,0.360352,-0.206909,-0.294678,-0.219238,-0.107422,5.551709,False
1,-0.109375,0.385742,-0.437500,-0.106445,0.232422,-0.273926,-0.333984,0.144531,0.205688,-0.072754,...,0.386719,0.303894,0.418945,0.106689,0.008789,0.203613,0.397949,-0.028381,5.354463,False
2,-0.494141,0.117798,0.272949,0.127930,0.374023,0.183105,-0.137695,-0.200439,-0.036346,0.378906,...,0.309082,-0.293884,0.640137,-0.087891,-0.448242,0.345703,-0.207520,-0.123779,4.695697,False
3,-0.142578,0.088379,-0.107300,0.243652,0.500977,0.161377,-0.187500,-0.351074,0.105469,0.010742,...,-0.075684,0.258202,0.279297,-0.016113,0.146118,0.406494,0.583984,0.094727,4.308543,False
4,0.001953,0.019348,-0.297607,-0.033203,-0.736328,0.467773,0.066406,0.133667,-0.243652,0.168945,...,-0.299805,0.354004,0.183105,-0.282715,-0.025391,0.602051,-0.449219,-0.274170,4.718472,False
5,-0.301025,0.156250,-0.045746,-0.521484,0.223022,-0.323730,-0.316406,-0.219727,0.183838,0.059570,...,0.074219,0.404785,0.371094,-0.646484,-0.136719,-0.054626,0.233459,0.282776,4.398509,False
6,0.214920,-0.186890,-0.052734,0.552002,0.012451,0.021240,0.493164,0.130371,-0.139404,-0.028320,...,0.119957,0.010254,0.104004,0.077148,-0.446777,0.166504,0.031250,-0.117554,3.311437,False
7,-0.016602,0.032349,-0.043152,0.013672,-0.045166,0.236816,0.118652,-0.065430,0.076599,0.111328,...,0.254883,0.121094,0.025391,-0.146973,0.238403,0.000183,-0.059082,-0.202148,2.904589,False
9,0.235352,-0.097046,0.477539,0.016602,-0.464844,0.182373,0.097168,-0.309570,-0.084473,0.038330,...,-0.040039,-0.452148,0.120850,0.007812,-0.083984,-0.144165,0.337891,-0.100586,4.724591,False
10,0.071657,0.064453,-0.406250,0.151855,-0.006836,-0.112305,-0.120117,0.408600,0.007812,-0.048828,...,-0.274902,-0.329102,0.197266,-0.074829,-0.276367,0.127197,0.083862,0.133789,3.674977,False


In [217]:

preds = clf.predict(X)

In [211]:
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])

precision 0.806451612903
recall 0.892857142857
f1 0.847457627119
True 0.892857142857
False 0.980456026059


In [202]:
p_hat = clf.coef_[0][301:]

In [212]:
def normalize(v):
  '''normalize' a vector, in the traditional linear algebra sense.'''
  norm=np.linalg.norm(v)
  if norm==0:
    return v
  return v/norm
def reject(A):

  '''Create a 'projection', and subract it from the original vector'''
  B = p_hat
  project = np.linalg.linalg.dot(A, normalize(B)) * normalize(B)
  return A - project


In [213]:
X_hat = X.apply(reject, axis=1)

ValueError: ('shapes (301,) and (0,) not aligned: 301 (dim 0) != 0 (dim 0)', u'occurred at index 0')

In [108]:
clf_hat = svm.LinearSVC()
clf_hat.fit(X_hat, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [109]:
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]
preds_hat = clf_hat.predict(X_test)
print "precision", metrics.precision_score(y_test, preds_hat)
print "recall", metrics.recall_score(y_test, preds_hat)
print "f1", metrics.f1_score(y_test, preds_hat)
print "True", metrics.accuracy_score(y_test[y_test == 1], preds_hat[y_test == 1])
print "False", metrics.accuracy_score(y_test[y_test == 0], preds_hat[y_test == 0])

precision 0.416243654822
recall 0.30258302583
f1 0.350427350427
True 0.30258302583
False 0.964788732394


In [110]:
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])

precision 0.806451612903
recall 0.892857142857
f1 0.847457627119
True 0.892857142857
False 0.980456026059


In [157]:
clf = joblib.load('models/svm.pkl') 

In [170]:
fp = 'lexical_entailment/bless2011/data_lex_test.tsv'
test = pd.read_csv(fp, sep='\t', header=None)
test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized.tsv', sep='\t', header=None)

In [171]:

test_df = test_vectorized
test_df.dropna(axis=0, inplace=True)

X = test_df.iloc[:, :-1]
y = test_df.iloc[:, -1]

preds = clf.predict(X)
true_count = y.sum()
preds_count = preds.sum()
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])
print true_count, preds_count, orig_rows

num_correct = metrics.accuracy_score(y, preds, normalize=False)

print test_name, ": percentage non-nan correct:", num_correct/float(test_df.shape[0])
print test_name, ": percentage correct overall", num_correct/float(orig_rows)

precision 0.406113537118
recall 0.343173431734
f1 0.372
True 0.343173431734
False 0.958358848745
271 229 358
val : percentage non-nan correct: 0.911224201301
val : percentage correct overall 9.00279329609


In [140]:
#Vectorize train/test/val sets

path_to_dataset = 'lexical_entailment/turney2014/data_lex_val.tsv'
path_to_vectorized = 'lexical_entailment/turney2014/data_lext_val_vectorized.tsv'
df = pd.read_csv(path_to_dataset, sep='\t', header=None)

words_0 = df[0]
words_1 = df[1]
y = df[[2]]
method = 'concat'
def vectorize_word(word):
	if word in model.wv:
		return pd.Series(model.wv[word])
	else:
		# The pretrained word2vec model has dimensionality 300
		return pd.Series([0] * 300)

vectors_0 = words_0.apply(vectorize_word)
vectors_1 = words_1.apply(vectorize_word)
def merge_vectors(v1, v2, method):
	print "method", method
	# Concat
	if method == 'concat':
		return pd.concat([vectors_0, vectors_1, y], axis = 1)
	elif method == 'diff':
		# Roller 2014 says to normalize the difference
		diff = vectors_0 - vectors_1
		return pd.concat([diff, y], axis = 1)
	elif method == 'asym':
		print "asym"
		# diff
		a = vectors_0 - vectors_1
		# squared diff - can't tell if they mean the mag^2 or ea element sq?
		b = pd.DataFrame(np.sqrt(np.square(a.values).sum(axis=1)))
		return pd.concat([a, b, y], axis = 1)
	elif method == 'cosine':
		#this fails???
		print "cosine"
		cos = pd.DataFrame(cosine_similarity(vectors_0.values, vectors_1.values).diagonal())
		return pd.concat([v1, v2, cos, y], axis = 1)

vectors_x = merge_vectors(vectors_0, vectors_1, method)
vectors_x.to_csv(path_to_vectorized, sep='\t', header=False, index=False)

method concat


In [151]:
X

,0,1
0,absentia,sentence
1,accounting,management
2,activist,federation
3,advisories,federation
4,aerospace,management
5,agency,government
6,agriculture,environment
7,agriculture,management
8,ambassador,chairman
9,amendment,bill
